In [2]:
!pip install pyswarm

  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4479 sha256=a447fac1361968800713fbeebe8026ba60935c4f57a1fb727860f17d4ba83f34
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ff\d2\b7\80118e5698de2bd0b8d1b3397abf7fdfc45c15ffc454b52145
Successfully built pyswarm


In this code, we first define the CNN model architecture, the loss function, and the accuracy metric. Then, we define the PSO optimizer function, which takes the CNN model parameters and the dataset as inputs and returns the negative of the validation accuracy (since PSO is a minimization algorithm).

Next, we define the bounds for the CNN parameters and load the CIFAR-10 dataset. Finally, we define the PSO algorithm parameters, such as the number of particles and maximum iterations, and run the PSO algorithm using the pso() function from the pyswarm library. The pso() function takes as

In [1]:
import numpy as np
import tensorflow as tf
from pyswarm import pso

# Define the CNN model architecture
def create_model(input_shape, num_classes, filters=32, kernel_size=(3,3), pool_size=(2,2)):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=pool_size),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define the loss function and the accuracy metric
def loss_function(y_true, y_pred):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return loss

def accuracy_metric(y_true, y_pred):
    correct_predictions = tf.equal(tf.argmax(y_true, axis=1), tf.argmax(y_pred, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy

# Define the PSO optimization function
def pso_optimizer(cnn_params, X_train, y_train, X_val, y_val):
    print(cnn_params)
    input_shape = X_train.shape[1:]
    num_classes = y_train.shape[1]
    model = create_model(input_shape, num_classes, filters=int(cnn_params[0]),
                         kernel_size=(int(cnn_params[1]), int(cnn_params[2])), 
                         pool_size=(int(cnn_params[3]), int(cnn_params[4])))
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_function, metrics=[accuracy_metric])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    _, val_acc = model.evaluate(X_val, y_val, verbose=1)
    return -val_acc

# Define the bounds for the CNN parameters
# bounds = [(16, 128), (3, 7), (3, 7), (2, 4), (2, 4)]
lb = [16,3,3,2,2]
ub = [128,7,7,4,4]
# Load the dataset
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_val = tf.keras.utils.to_categorical(y_val, 10)

# Define the PSO algorithm parameters
n_particles = 10
max_iter = 20

# Run the PSO algorithm to optimize the CNN model
# best_cost, best_params = pso(pso_optimizer, bounds, args=(X_train, y_train, X_val, y_val), swarmsize=n_particles, maxiter=max_iter)
best_cost, best_params = pso(pso_optimizer, lb,ub, args=(X_train, y_train, X_val, y_val), swarmsize=n_particles, maxiter=max_iter)

# Print the best CNN model's parameters and validation accuracy
print("Best CNN parameters: ", best_params)
print("Best validation accuracy: ", -best_cost)


[103.23350192   4.40664845   6.16729221   2.67368455   3.96048844]
Epoch 1/10
1563/1563 [==============================] - 6s 2ms/step - loss: 1.4833 - accuracy_metric: 0.4754 - val_loss: 1.3210 - val_accuracy_metric: 0.5316
Epoch 2/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.1921 - accuracy_metric: 0.5885 - val_loss: 1.1532 - val_accuracy_metric: 0.5933
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0738 - accuracy_metric: 0.6301 - val_loss: 1.1354 - val_accuracy_metric: 0.6115
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9984 - accuracy_metric: 0.6569 - val_loss: 1.0821 - val_accuracy_metric: 0.6275
Epoch 5/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9476 - accuracy_metric: 0.6735 - val_loss: 1.0640 - val_accuracy_metric: 0.6387
Epoch 6/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9021 - accuracy_metric: 0.6893 - val_loss: 1.0718 - val_accuracy_

Epoch 5/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.1007 - accuracy_metric: 0.6177 - val_loss: 1.1560 - val_accuracy_metric: 0.6032
Epoch 6/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.0645 - accuracy_metric: 0.6331 - val_loss: 1.1811 - val_accuracy_metric: 0.5966
Epoch 7/10
1563/1563 [==============================] - 3s 2ms/step - loss: 1.0323 - accuracy_metric: 0.6421 - val_loss: 1.1450 - val_accuracy_metric: 0.6050
Epoch 8/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0023 - accuracy_metric: 0.6539 - val_loss: 1.1454 - val_accuracy_metric: 0.6090
Epoch 9/10
1563/1563 [==============================] - 3s 2ms/step - loss: 0.9840 - accuracy_metric: 0.6605 - val_loss: 1.1601 - val_accuracy_metric: 0.6035
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 1.1214 - accuracy_metric: 0.6188
[128.           3.35232465   6.01458856   2.02513392   3.01335834]
Epoch 1/10
1563/1563 [=============

Epoch 2/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.1757 - accuracy_metric: 0.5947 - val_loss: 1.1457 - val_accuracy_metric: 0.5991
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0465 - accuracy_metric: 0.6395 - val_loss: 1.1032 - val_accuracy_metric: 0.6225
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9547 - accuracy_metric: 0.6705 - val_loss: 1.1164 - val_accuracy_metric: 0.6134
Epoch 5/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8820 - accuracy_metric: 0.6956 - val_loss: 1.0920 - val_accuracy_metric: 0.6251
Epoch 6/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8217 - accuracy_metric: 0.7162 - val_loss: 1.1002 - val_accuracy_metric: 0.6351
Epoch 7/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.7562 - accuracy_metric: 0.7381 - val_loss: 1.1328 - val_accuracy_metric: 0.6315
Epoch 8/10
1563/1563 [==============================

Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 1.1197 - accuracy_metric: 0.6210
[17.98852098  7.          6.38577001  2.74929527  3.53289803]
Epoch 1/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.5653 - accuracy_metric: 0.4469 - val_loss: 1.3682 - val_accuracy_metric: 0.5230
Epoch 2/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.3103 - accuracy_metric: 0.5412 - val_loss: 1.2904 - val_accuracy_metric: 0.5511
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.2231 - accuracy_metric: 0.5752 - val_loss: 1.2463 - val_accuracy_metric: 0.5603
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.1692 - accuracy_metric: 0.5953 - val_loss: 1.2152 - val_accuracy_metric: 0.5793
Epoch 5/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.1268 - accuracy_metric: 0.6120 - val_loss: 1.1445 - val_accuracy_metric: 0.6058
Epoch 6/10
1563/1563 [==================

Epoch 7/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.8682 - accuracy_metric: 0.7006 - val_loss: 0.9760 - val_accuracy_metric: 0.6666
Epoch 8/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8392 - accuracy_metric: 0.7122 - val_loss: 0.9854 - val_accuracy_metric: 0.6630
Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8122 - accuracy_metric: 0.7219 - val_loss: 0.9907 - val_accuracy_metric: 0.6664
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 1.0098 - accuracy_metric: 0.6576
[127.82043084   4.98936769   3.41937444   2.53135506   2.36489826]
Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4400 - accuracy_metric: 0.4927 - val_loss: 1.3509 - val_accuracy_metric: 0.5358
Epoch 2/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.1592 - accuracy_metric: 0.6002 - val_loss: 1.1360 - val_accuracy_metric: 0.6097
Epoch 3/10
1563/1563 [=============

...

313/313 [==============================] - 1s 2ms/step - loss: 1.0195 - accuracy_metric: 0.6569
[119.66187015   4.0783911    3.73219393   4.           3.42903624]
Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4494 - accuracy_metric: 0.4916 - val_loss: 1.2763 - val_accuracy_metric: 0.5664
Epoch 2/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.1725 - accuracy_metric: 0.5955 - val_loss: 1.1593 - val_accuracy_metric: 0.6060
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0721 - accuracy_metric: 0.6315 - val_loss: 1.0559 - val_accuracy_metric: 0.6334
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0077 - accuracy_metric: 0.6550 - val_loss: 1.0421 - val_accuracy_metric: 0.6446
Epoch 5/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9541 - accuracy_metric: 0.6741 - val_loss: 1.0169 - val_accuracy_metric: 0.6570
Epoch 6/10
1563/1563 [=========================

Epoch 8/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.8322 - accuracy_metric: 0.7151 - val_loss: 0.9851 - val_accuracy_metric: 0.6646
Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8091 - accuracy_metric: 0.7222 - val_loss: 0.9806 - val_accuracy_metric: 0.6740
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.9779 - accuracy_metric: 0.6723
[119.6885404    4.04703854   3.70271898   4.           3.41792823]
Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4325 - accuracy_metric: 0.4984 - val_loss: 1.2435 - val_accuracy_metric: 0.5765
Epoch 2/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.1613 - accuracy_metric: 0.5999 - val_loss: 1.1513 - val_accuracy_metric: 0.6092
Epoch 3/10
1563/1563 [==============================] - 4s 2ms/step - loss: 1.0653 - accuracy_metric: 0.6333 - val_loss: 1.0769 - val_accuracy_metric: 0.6354
Epoch 4/10
1563/1563 [=============

Epoch 5/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.9581 - accuracy_metric: 0.6707 - val_loss: 1.0126 - val_accuracy_metric: 0.6499
Epoch 6/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.9173 - accuracy_metric: 0.6866 - val_loss: 1.0539 - val_accuracy_metric: 0.6416
Epoch 7/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.8738 - accuracy_metric: 0.6995 - val_loss: 1.0445 - val_accuracy_metric: 0.6512
Epoch 8/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8440 - accuracy_metric: 0.7118 - val_loss: 0.9727 - val_accuracy_metric: 0.6724
Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8209 - accuracy_metric: 0.7166 - val_loss: 1.0161 - val_accuracy_metric: 0.6542
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.9986 - accuracy_metric: 0.6633
[119.68877071   4.06141475   3.71590072   4.           3.42488133]
Epoch 1/10
1563/1563 [=============

Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8132 - accuracy_metric: 0.7208 - val_loss: 1.0137 - val_accuracy_metric: 0.6549
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 1.0090 - accuracy_metric: 0.6550
[120.04170034   4.02776936   3.75688063   4.           3.47554205]
Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4470 - accuracy_metric: 0.4912 - val_loss: 1.2385 - val_accuracy_metric: 0.5712
Epoch 2/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.1790 - accuracy_metric: 0.5933 - val_loss: 1.2419 - val_accuracy_metric: 0.5567
Epoch 3/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.0764 - accuracy_metric: 0.6281 - val_loss: 1.0958 - val_accuracy_metric: 0.6273
Epoch 4/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9999 - accuracy_metric: 0.6569 - val_loss: 1.0385 - val_accuracy_metric: 0.6431
Epoch 5/10
1563/1563 [=============

Epoch 6/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.9059 - accuracy_metric: 0.6881 - val_loss: 0.9940 - val_accuracy_metric: 0.6593
Epoch 7/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8703 - accuracy_metric: 0.7014 - val_loss: 1.0598 - val_accuracy_metric: 0.6410
Epoch 8/10
1563/1563 [==============================] - 4s 3ms/step - loss: 0.8387 - accuracy_metric: 0.7127 - val_loss: 0.9832 - val_accuracy_metric: 0.6596
Epoch 9/10
1563/1563 [==============================] - 4s 2ms/step - loss: 0.8079 - accuracy_metric: 0.7216 - val_loss: 1.0102 - val_accuracy_metric: 0.6588
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.9749 - accuracy_metric: 0.6719
[119.89176324   4.05177847   3.71308356   4.           3.42236063]
Epoch 1/10
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4440 - accuracy_metric: 0.4928 - val_loss: 1.2719 - val_accuracy_metric: 0.5508
Epoch 2/10
1563/1563 [=============